# Link Prediction with Tensor Factorization (VecHGrad method)

*Last edited: 2021-01-01*

### Based on work from

Gao, Sheng, Ludovic Denoyer, and Patrick Gallinari. "Link pattern prediction with tensor decomposition in multi-relational networks." In 2011 IEEE Symposium on Computational Intelligence and Data Mining (CIDM), pp. 333-340. IEEE, 2011.

*Improvements in tensor decomposition from:*

VecHGrad: https://arxiv.org/abs/1905.12413

VecHGrad Code: https://github.com/dagrate/vechgrad

### Data

* bowling-green.american-assembly from https://github.com/pol-is/openData
  * Filled in with 0 for missing data
  * Split in two matrices, one with positive observations (1) and one with negative observations (-1).

### How to use

* Matrices with pos-neg layer data need to be uploaded to Google Colab
* Vector *a* (line 6 below) needs to be updated with dimensions of input data
  * Can also be extracted by the csv files themselves
* Run helper function cells and main program
* Resulting latent feature matrices A, B and C are the ones that minimize the weighted least squares error for the missing data.

### Technical details

* This notebook uses the [TensorLy](http://tensorly.org/stable/index.html) Python module for tensor representation.
* TensorLy supports all of NumPy, Pytorch and Tensorflow as backends (among others), and the notebook is written in a way that supports all three.
  * Just change the backend in the `tl.set_backend('pytorch')` line.
* Runtime can be changed to GPU or TPU through the menu option *Runtime -> Change runtime type*. (Currently set to *None*)
* Some quick performance comparisons (nr_of_iter=1000, random.seed=2320) :
  * NumPy: 161.222 s (None)
  * PyTorch: 13.9823 s (None), 4.65814 s (GPU), 5.08854 s (TPU)
  * TensorFlow: 16.4367 s (None), 2.8769 s (GPU), 13.6554 s (TPU)


## Notebook initialization

In [15]:
!pip install tensorly
import numpy as np
import tensorly as tl
!pip install line_profiler
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


## Helper functions

In [16]:
### TNSRUTILS.JI FUNCTIONS

# Weighted least squares error
def objfunweighted(X, Xh, Wt):
  # Objective function for Least Square Error for any backend 
  return tl.backend.sqrt(tl.backend.sum((Wt * X - Wt * Xh) ** 2))

### ALSCP.JI FUNCTION

def nncpals(X, rankR, A, B, C, Weights, maxiter=1000, epsobjfun=1.0E-3):
    cp_weights = tl.tensor(np.array([1., 1.])) # Weights needed for rebuilding X from A,B,C

    if len(X.shape) > 3:
        return println("Only third order parafac implemented") ;

    # initialization with supplied random values
    Xh = tl.cp_to_tensor((cp_weights,[A,B,C]))  #Equiv in Tensorly

    # non-negative CP ALS iterative process
    i = 1 ;
    while i <= maxiter:
        for mode in range(len(X.shape)):

            # Hadamard product on all mode != n
            V = tl.ones((rankR, rankR)) ;
            for n in range(len(X.shape)):
              if n != mode:
                if n == 0:
                  tmpV = A ;
                elif n == 1:
                  tmpV = B ;
                elif n == 2:
                  tmpV = C ;
                V = V * (tl.transpose(tmpV) @ tmpV) ;

            # Khatri-Rao product on all mode != n
            if mode == 0:
              W = tl.kr((C, B)) ;
            elif mode == 1:
              W = tl.kr((C, A)) ;
            elif mode == 2:
              W = tl.kr((B, A)) ;

            # non negative update
            num = (tl.unfold(X, mode) @ W) + 1.0E-9 ;
            if mode == 0:
              denum = (A @ (tl.transpose(W) @ W)) + 1.0E-9 ;
              A = A * (num / denum);
            elif mode == 1:
              denum = (B @ (tl.transpose(W) @ W)) + 1.0E-9 ;
              B = B * (num / denum);
            elif mode == 2:
              denum = (C @ (tl.transpose(W) @ W)) + 1.0E-9 ;
              C = C * (num / denum);

        # calculation evolution
        Xh = tl.cp_to_tensor((cp_weights,[A,B,C]))  #Equiv in Tensorly
        curvo = objfunweighted(X,Xh,Weights) ;
        if curvo > epsobjfun:
          i += 1 ;
        else:
          i = maxiter + 1 ;
  
    print("Number of iterations: " + str(i-1)) ;
    Xh = tl.cp_to_tensor((cp_weights,[A,B,C]))  #Equiv in Tensorly

    return A, B, C ;

## Main program

In [18]:
a = (2031, 896, 2) ;  # Participant-votes data, 2 layers (pos,neg)
maxitrtn = 1000 ;     # Nr of iterations
flpath = "/content/"  # Path to data files
latfact = 2
np.random.seed(2320)  # Fix seed for backend comparison

tl.set_backend('tensorflow') # Change backend used by TensorLy

# Assembling the participant-votes tensor
Weights = tl.tensor(np.random.choice(a=[0., 1.], size=(a[0], a[1], a[2]) ));   # 1 for observations, 0 for missing links (to be predicted)
X = tl.tensor(np.stack([np.genfromtxt(flpath + "participant-votes-dataonly-positive.csv", delimiter=','),
                        np.genfromtxt(flpath + "participant-votes-dataonly-negative.csv", delimiter=',')], 
                       axis=2
                      )
             )

# resolution of CP
A, B, C = tl.tensor(np.random.rand(a[0], latfact)), tl.tensor(np.random.rand(a[1], latfact)), tl.tensor(np.random.rand(a[2], latfact))
#A, B, C = nncpals(X, latfact, A, B, C, Weights, maxitrtn) ;
%lprun -f nncpals nncpals(X, latfact, A, B, C, Weights, 1000) ;  # Profiling main function

Number of iterations: 1000


# Experiments / speedups

## Comparing naive rebuilding X from A,B,C vs specialized tensorly.cp_to_tensor command

In [ ]:
#from tensorly.decomposition import parafac
#weights, factors = parafac(X, rank=2)
weights = np.array([1., 1.])
TL = tl.cp_to_tensor((weights,[A,B,C]))
Xh = buildCP3(A,B,C)
TL - Xh

## Profiling errorfun

In [ ]:
weights = np.array([1., 1.])
Xh = tl.cp_to_tensor((weights,[A,B,C]))
%lprun -f errorfun errorfun(X,Xh)
#%lprun -f tl.metrics.regression.MSE tl.metrics.regression.MSE(X,Xh)

In [ ]:
weights = np.array([1., 1.])
Xh = tl.cp_to_tensor((weights,[A,B,C]))
%lprun -f objfunweighted objfunweighted(X, Xh, Weights)

## Naive check on how many participant-votes observations are predicted both positive and negative (obv a mistake)


In [ ]:
'''
Xh = buildCP3(A, B, C) ;
counter = 0
for i = 1:size(Xh)[1]
  for j = 1:size(Xh)[2]
    if Xh[i,j,1] > 0.5 && Xh[i,j,2] < -0.5
      counter += 1 ;
    end
  end
end
print("Mistakes: ")
print(counter) ;
print(" out of ")
print(size(Xh)[1] * size(Xh)[2]) ;
println(" (", 100 * counter / (size(Xh)[1] * size(Xh)[2]), "%)") ;
'''

## Obsolete code

In [ ]:
def objfunweighted(X, Xh, W):
  #Z = np.multiply(X-Xh,X-Xh) ;
  Z = (X-Xh) **2 
  for l in range(Z.shape[2]):
    Z[:,:,l] = np.multiply(Z[:,:,l], W) ;

  return np.sqrt( np.sum(Z) )

In [ ]:
def objfunweighted(X, Xh, Wt):
  zrs = tl.zeros_like(X)
  weighted_X = tl.where(Wt == 1, X, zrs)
  weighted_Xh = tl.where(Wt == 1, Xh, zrs)
  
  return np.sqrt(np.sum((weighted_X - weighted_Xh)**2))